In [0]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/FileStore/tables/empmysql1.csv")
df.show()

+-----+------+---------+----+-----------+----+----+------+------+---+
|empno| ename|      job| mgr|   hiredate| sal|comm|deptno|gender|age|
+-----+------+---------+----+-----------+----+----+------+------+---+
| 7369| SMITH|    CLERK|7902|17-Dec-1980| 800|  20|    10|     f| 22|
| 7499| ALLEN| SALESMAN|7698|20-Feb-1981|1500| 300|    30|     f| 30|
| 7521|  WARD| SALESMAN|7698|22-Feb-1981|1250| 500|    30|     m| 40|
| 7521|  WARD| SALESMAN|7698|22-Feb-1981|1250| 500|    30|     f| 90|
| 7566| JONES|  MANAGER|7839| 2-Apr-1981|2975|   0|    20|     m| 20|
| 7654|MARTIN| SALESMAN|7698|28-Sep-1981|1150|1400|    30|     f| 50|
| 7698| BLAKE|  MANAGER|7839| 1-May-1981|2850|   0|    30|     f| 28|
| 7782| CLARK|  MANAGER|7839| 9-Jun-1981|2440|   0|    10|     m| 50|
| 7788|  null|  ANALYST|7566|19-Apr-1987|3000|   0|    20|     f| 22|
| 7789|  VENU|  ANALYST|7506|19-Apr-1989|3100|   0|    20|     f| 19|
| 7839|  VENU|PRESIDENT|null|17-Nov-1981|5000|null|    10|     f| 89|
| 7844|TURNER| SALES

In [0]:
df.createOrReplaceTempView("tab")
res=spark.sql("select *,dense_rank()over(order by sal desc) drank from tab")
from pyspark.sql.window import *
from pyspark.sql.functions import *
win= Window.orderBy(col("sal").desc())
ndf=df.withColumn("drank",dense_rank().over(win)).withColumn("rnk",rank().over(win)).withColumn("rno",row_number().over(win))
display(ndf) 

empno,ename,job,mgr,hiredate,sal,comm,deptno,gender,age,drank,rnk,rno
7839,VENU,PRESIDENT,null,17-Nov-1981,5000,null,10,f,89,1,1,1
7789,VENU,ANALYST,7506,19-Apr-1989,3100,0,20,f,19,2,2,2
7788,null,ANALYST,7566,19-Apr-1987,3000,0,20,f,22,3,3,3
7566,JONES,MANAGER,7839,2-Apr-1981,2975,0,20,m,20,4,4,4
7698,BLAKE,MANAGER,7839,1-May-1981,2850,0,30,f,28,5,5,5
7782,CLARK,MANAGER,7839,9-Jun-1981,2440,0,10,m,50,6,6,6
7499,ALLEN,SALESMAN,7698,20-Feb-1981,1500,300,30,f,30,7,7,7
7844,TURNER,SALESMAN,7698,8-Sep-1981,1500,0,30,m,40,7,7,8
7845,TURNER,SALESMAN,7698,8-Sep-1981,1500,0,30,f,40,7,7,9
7521,WARD,SALESMAN,7698,22-Feb-1981,1250,500,30,m,40,8,10,10


In [0]:
#ntile is used for grading system or distinction type of system 
#percentage rank is used to give rank between 0 and 1
ndf1=ndf.withColumn("ntile",ntile(3).over(win)).withColumn("per",percent_rank().over(win))
display(ndf1)

empno,ename,job,mgr,hiredate,sal,comm,deptno,gender,age,drank,rnk,rno,ntile,per
7839,VENU,PRESIDENT,null,17-Nov-1981,5000,null,10,f,89,1,1,1,1,0.0
7789,VENU,ANALYST,7506,19-Apr-1989,3100,0,20,f,19,2,2,2,1,0.07692307692307693
7788,null,ANALYST,7566,19-Apr-1987,3000,0,20,f,22,3,3,3,1,0.15384615384615385
7566,JONES,MANAGER,7839,2-Apr-1981,2975,0,20,m,20,4,4,4,1,0.23076923076923078
7698,BLAKE,MANAGER,7839,1-May-1981,2850,0,30,f,28,5,5,5,1,0.3076923076923077
7782,CLARK,MANAGER,7839,9-Jun-1981,2440,0,10,m,50,6,6,6,2,0.38461538461538464
7499,ALLEN,SALESMAN,7698,20-Feb-1981,1500,300,30,f,30,7,7,7,2,0.46153846153846156
7844,TURNER,SALESMAN,7698,8-Sep-1981,1500,0,30,m,40,7,7,8,2,0.46153846153846156
7845,TURNER,SALESMAN,7698,8-Sep-1981,1500,0,30,f,40,7,7,9,2,0.46153846153846156
7521,WARD,SALESMAN,7698,22-Feb-1981,1250,500,30,m,40,8,10,10,2,0.6923076923076923
